In [21]:
import json

with open('E:/anno.json', 'rt') as in_file:
    content = json.load(in_file)

In [25]:
content.keys()

dict_keys(['metadata', 'samples'])

In [26]:
content['metadata']

{'description': 'UA-DETRAC benchmark dataset XML ingestion',
 'key_hash': 0,
 'last_modified': '2021-10-01 15:50:18.133952',
 'version': '1.0.0.'}

In [29]:
content['samples'].keys()

dict_keys(['MVI_20011', 'MVI_20012', 'MVI_20032', 'MVI_20033', 'MVI_20034', 'MVI_20035', 'MVI_20051', 'MVI_20052', 'MVI_20061', 'MVI_20062', 'MVI_20063', 'MVI_20064', 'MVI_20065', 'MVI_39031', 'MVI_39051', 'MVI_39211', 'MVI_39271', 'MVI_39311', 'MVI_39361', 'MVI_39371', 'MVI_39401', 'MVI_39501', 'MVI_39511', 'MVI_39761', 'MVI_39771', 'MVI_39781', 'MVI_39801', 'MVI_39811', 'MVI_39821', 'MVI_39851', 'MVI_39861', 'MVI_39931', 'MVI_40131', 'MVI_40141', 'MVI_40152', 'MVI_40161', 'MVI_40162', 'MVI_40171', 'MVI_40172', 'MVI_40181', 'MVI_40191', 'MVI_40192', 'MVI_40201', 'MVI_40204', 'MVI_40211', 'MVI_40212', 'MVI_40213', 'MVI_40241', 'MVI_40243', 'MVI_40244', 'MVI_40701', 'MVI_40711', 'MVI_40712', 'MVI_40714', 'MVI_40732', 'MVI_40742', 'MVI_40743', 'MVI_40751', 'MVI_40752', 'MVI_40761', 'MVI_40762', 'MVI_40763', 'MVI_40771', 'MVI_40772', 'MVI_40773', 'MVI_40774', 'MVI_40775', 'MVI_40792', 'MVI_40793', 'MVI_40851', 'MVI_40852', 'MVI_40853', 'MVI_40854', 'MVI_40855', 'MVI_40863', 'MVI_40864', '

In [31]:
sample = content['samples']['MVI_20011']
sample

{'entities': [{'bb': [593, 379, 160, 162],
   'blob': {'color': 'Black',
    'frame_idx': 0,
    'frame_xml': 1,
    'orientation': 18.488,
    'speed': 6.859,
    'trajectory_length': 5.0,
    'truncation_ratio': 0.1,
    'vehicle_type': 'Suv'},
   'confidence': 1.0,
   'id': 1,
   'labels': {'Suv': 1},
   'time': 0},
  {'bb': [558, 121, 47, 43],
   'blob': {'color': 'White',
    'frame_idx': 0,
    'frame_xml': 1,
    'orientation': 19.398,
    'speed': 1.5055,
    'trajectory_length': 72.0,
    'truncation_ratio': 0.0,
    'vehicle_type': 'Sedan'},
   'confidence': 1.0,
   'id': 2,
   'labels': {'Sedan': 1},
   'time': 0},
  {'bb': [545, 88, 35, 30],
   'blob': {'color': 'Gray',
    'frame_idx': 0,
    'frame_xml': 1,
    'occlusion_box': [553, 88, 27, 2],
    'occlusion_status': 1,
    'orientation': 2.7525,
    'speed': 0.5206,
    'trajectory_length': 105.0,
    'truncation_ratio': 0.0,
    'vehicle_type': 'Sedan'},
   'confidence': 1.0,
   'id': 3,
   'labels': {'1': 1, 'Sedan':